# Run PESTPP-OPT

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import pyemu


flopy is installed in /Users/jeremyw/Dev/flopy/flopy


In [3]:
t_d = "template"
m_d = "master_opt"

In [5]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.write_par_summary_table(filename="none").sort_index()

,type,transform,count,initial value,upper bound,lower bound,standard deviation
drncond_k00,drncond_k00,log,10,0,1,-1,0.5
flow,flow,log,1,0,0.09691,-0.124939,0.0554622
grhk3,grhk3,log,705,0,1,-1,0.5
grhk4,grhk4,log,705,0,1,-1,0.5
grhk5,grhk5,log,705,0,1,-1,0.5
grrech2,grrech2,log,705,0,0.0413927,-0.0457575,0.0217875
grrech3,grrech3,log,705,0,0.0413927,-0.0457575,0.0217875
grss3,grss3,log,705,0,1,-1,0.5
grss4,grss4,log,705,0,1,-1,0.5
grss5,grss5,log,705,0,1,-1,0.5


In [6]:
pst.pestpp_options = {}
dvg = "wellflux_k02"
pst.pestpp_options["opt_dec_var_groups"] = dvg


In [15]:
par = pst.parameter_data
par.loc[:,"partrans"] = "fixed"
par.loc[par.pargp==dvg,"partrans"] = "none"
par.loc[par.pargp==dvg,"parlbnd"] = 0.0
par.loc[par.pargp==dvg,"parubnd"] = 3.0




In [16]:
obs = pst.observation_data
swgw_hist = obs.loc[obs.obsnme.apply(lambda x: "fa" in x and "1979" in x and("hw" in x or "tw" in x)),"obsnme"]
obs.loc[swgw_hist,:]

,obsnme,obsval,weight,obgnme,extra
obsnme,,,,,
fa_hw_19791230,fa_hw_19791230,-1000.0,0.0,less_than,NaN
fa_tw_19791230,fa_tw_19791230,-1000.0,0.0,less_than,NaN


We need to change the obs group (`obgnme`) so that `pestpp-opt` will recognize these two model outputs as constraints.  lets also assume that the sw-gw flux needs to be at least -1000

In [17]:
obs.loc[swgw_hist,"obgnme"] = "less_than"
obs.loc[swgw_hist[0],"obsval"] = -1000
obs.loc[swgw_hist[1],"obsval"] = -1000


In [18]:
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt.pst"))

In [19]:
pyemu.os_utils.start_slaves(t_d,"pestpp-opt","freyberg_opt.pst",num_slaves=5,master_dir=m_d)